## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

c:\Users\jaypa\AppData\Local\Programs\Python\Python39\lib\site-packages\requests\__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.7) or chardet (None)/charset_normalizer (3.3.2) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df.drop(columns=["EIN", "NAME"], inplace=True)

In [3]:
# Determine the number of unique values in each column.
unique_values = application_df.nunique()
unique_values

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [4]:
# Look at APPLICATION_TYPE value counts to identify and replace with "Other"
application_df["APPLICATION_TYPE"].value_counts()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [5]:
# Determine a cutoff point to combine "rare" categorical variables into "Other"
unique_values = application_df['APPLICATION_TYPE'].value_counts()
cutoff = 160
application_types_to_replace = unique_values[unique_values < cutoff].index

# Replace rare categories in APPLICATION_TYPE
application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(application_types_to_replace, "Other")

# Check to make sure replacement was successful
application_type_counts_after = application_df['APPLICATION_TYPE'].value_counts()
print(application_type_counts_after)

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64


In [6]:
# Look at CLASSIFICATION value counts to identify and replace with "Other"
classification_value_counts = application_df["CLASSIFICATION"].value_counts()
classification_value_counts

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [7]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classification_value_counts_greater_than_1 = classification_value_counts[classification_value_counts > 1]

classification_value_counts_greater_than_1

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [8]:
# Determine a cutoff point to combine "rare" categorical variables into "Other"
unique_values = application_df['CLASSIFICATION'].value_counts()
cutoff = 1000
classification_to_replace = unique_values[unique_values < cutoff].index

# Replace rare categories in APPLICATION_TYPE
application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(classification_to_replace, "Other")

# Check to make sure replacement was successful
classification_counts_after = application_df['CLASSIFICATION'].value_counts()
print(classification_counts_after)

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64


In [9]:
# Convert categorical data to numeric with `pd.get_dummies`
charity_data_encoded = pd.get_dummies(application_df, columns=['APPLICATION_TYPE', 'AFFILIATION', 'CLASSIFICATION', 'USE_CASE', 'ORGANIZATION', 'INCOME_AMT', 'SPECIAL_CONSIDERATIONS'])


In [10]:
# Split our preprocessed data into our features and target arrays
X = charity_data_encoded.drop('IS_SUCCESSFUL', axis=1)
y = charity_data_encoded['IS_SUCCESSFUL']

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [11]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [12]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

nn = tf.keras.models.Sequential()
input_features = X_train_scaled.shape[1]

# First hidden layer
nn.add(tf.keras.layers.Dense(units=80, input_dim=input_features, activation='relu'))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=30, activation='relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 80)                3520      
                                                                 
 dense_1 (Dense)             (None, 30)                2430      
                                                                 
 dense_2 (Dense)             (None, 1)                 31        
                                                                 
Total params: 5,981
Trainable params: 5,981
Non-trainable params: 0
_________________________________________________________________


In [13]:
# Compile the model
nn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [14]:
# Train the model
# Step 8: Create a callback that saves the model's weights every five epochs
checkpoint = tf.keras.callbacks.ModelCheckpoint('AlphabetSoupCharity.h5', save_weights_only=False, save_freq='epoch', period=5)

# Step 7: Train the model
history = nn.fit(X_train_scaled, y_train, epochs=100, batch_size=32, validation_split=0.2, callbacks=[checkpoint])

Epoch 1/100
686/686 [==============================] - 2s 2ms/step - loss: 0.5701 - accuracy: 0.7205 - val_loss: 0.5502 - val_accuracy: 0.7343
Epoch 2/100
686/686 [==============================] - 1s 2ms/step - loss: 0.5557 - accuracy: 0.7271 - val_loss: 0.5510 - val_accuracy: 0.7362
Epoch 3/100
686/686 [==============================] - 1s 2ms/step - loss: 0.5529 - accuracy: 0.7275 - val_loss: 0.5462 - val_accuracy: 0.7392
Epoch 4/100
686/686 [==============================] - 1s 2ms/step - loss: 0.5513 - accuracy: 0.7308 - val_loss: 0.5444 - val_accuracy: 0.7374
Epoch 5/100
686/686 [==============================] - 1s 2ms/step - loss: 0.5492 - accuracy: 0.7312 - val_loss: 0.5469 - val_accuracy: 0.7407
Epoch 6/100
686/686 [==============================] - 1s 2ms/step - loss: 0.5487 - accuracy: 0.7291 - val_loss: 0.5438 - val_accuracy: 0.7409
Epoch 7/100
686/686 [==============================] - 1s 2ms/step - loss: 0.5480 - accuracy: 0.7304 - val_loss: 0.5453 - val_accuracy: 0.7414

In [15]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

215/215 - 0s - loss: 0.5637 - accuracy: 0.7245 - 174ms/epoch - 809us/step
Loss: 0.5637401938438416, Accuracy: 0.7244898080825806


In [17]:
# Export our model to HDF5 file
nn.save('AlphabetSoupCharity.h5')